<a href="https://colab.research.google.com/github/ACRainieri/pyspark/blob/main/Mini_projeto_DSA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q imdb-sqlite
!pip install -q pycountry

In [ ]:
import re
import time
import sqlite3
import pycountry
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import cm
from sklearn.feature_extraction.text import CountVectorizer
import warnings 

warnings.filterwarnings('ignore')
sns.set_theme(style= 'whitegrid')


In [ ]:
%%time
!imdb-sqlite

2022-09-13 13:35:21,345 GET https://datasets.imdbws.com/name.basics.tsv.gz -> downloads/name.basics.tsv.gz
2022-09-13 13:35:22,309 GET https://datasets.imdbws.com/title.basics.tsv.gz -> downloads/title.basics.tsv.gz
2022-09-13 13:35:22,935 GET https://datasets.imdbws.com/title.akas.tsv.gz -> downloads/title.akas.tsv.gz
2022-09-13 13:35:24,058 GET https://datasets.imdbws.com/title.principals.tsv.gz -> downloads/title.principals.tsv.gz
2022-09-13 13:35:25,668 GET https://datasets.imdbws.com/title.episode.tsv.gz -> downloads/title.episode.tsv.gz
2022-09-13 13:35:25,827 GET https://datasets.imdbws.com/title.ratings.tsv.gz -> downloads/title.ratings.tsv.gz
2022-09-13 13:35:25,893 Populating database: imdb.db
2022-09-13 13:35:25,893 Applying schema
2022-09-13 13:35:25,895 Importing file: downloads/name.basics.tsv.gz
2022-09-13 13:35:25,895 Reading number of rows ...
2022-09-13 13:35:32,304 Inserting rows into table: people
100% 11921620/11921620 [02:11<00:00, 90841.16 rows/s]
2022-09-13 13:3

In [ ]:
conn = sqlite3.connect('imdb.db')

In [ ]:
tabelas = pd.read_sql_query("SELECT NAME as 'Table_Name' FROM sqlite_master WHERE type = 'table'", conn)

tabelas.head()  
tabelas.Table_Name


0      people
1      titles
2        akas
3        crew
4    episodes
5     ratings
Name: Table_Name, dtype: object

In [ ]:
tabelas = tabelas["Table_Name"].values.tolist()
tabelas

['people', 'titles', 'akas', 'crew', 'episodes', 'ratings']

In [ ]:
for tabela in tabelas:
  #consulta = "PRAGMA_TABLE_INFO({})".format(tabela)
  consulta = 'PRAGMA table_info(' + tabela +')'
  resultado = pd.read_sql_query(consulta, conn)
  print("Esquema da Tabela: "+ tabela)
  display(resultado)
 

Esquema da Tabela: people


,cid,name,type,notnull,dflt_value,pk
0,0,person_id,VARCHAR,0,None,1
1,1,name,VARCHAR,0,None,0
2,2,born,INTEGER,0,None,0
3,3,died,INTEGER,0,None,0


Esquema da Tabela: titles


,cid,name,type,notnull,dflt_value,pk
0,0,title_id,VARCHAR,0,None,1
1,1,type,VARCHAR,0,None,0
2,2,primary_title,VARCHAR,0,None,0
3,3,original_title,VARCHAR,0,None,0
4,4,is_adult,INTEGER,0,None,0
5,5,premiered,INTEGER,0,None,0
6,6,ended,INTEGER,0,None,0
7,7,runtime_minutes,INTEGER,0,None,0
8,8,genres,VARCHAR,0,None,0


Esquema da Tabela: akas


...

,cid,name,type,notnull,dflt_value,pk
0,0,title_id,VARCHAR,0,None,0
1,1,person_id,VARCHAR,0,None,0
2,2,category,VARCHAR,0,None,0
3,3,job,VARCHAR,0,None,0
4,4,characters,VARCHAR,0,None,0


Esquema da Tabela: episodes


,cid,name,type,notnull,dflt_value,pk
0,0,episode_title_id,INTEGER,0,None,0
1,1,show_title_id,INTEGER,0,None,0
2,2,season_number,INTEGER,0,None,0
3,3,eposide_number,INTEGER,0,None,0


Esquema da Tabela: ratings


,cid,name,type,notnull,dflt_value,pk
0,0,title_id,VARCHAR,0,None,1
1,1,rating,INTEGER,0,None,0
2,2,votes,INTEGER,0,None,0


Ajustando um Dataset, agrupando em outros os valores menores de cinco porcento o restante da lista.

In [ ]:
consulta1 = '''SELECT type, COUNT(*) as COUNT FROM titles group by type'''
resultado1 = pd.read_sql_query(consulta1, conn)

resultado1['percentual'] = (resultado1['COUNT'] / resultado1['COUNT'].sum()) * 100

others = {}

others['COUNT'] = resultado1[resultado1['percentual'] < 5] ['percentual'].sum()

others['type'] = 'others'

#display(resultado1)
display(others)

#filtra o dataframe de resultado
resultado1 = resultado1[resultado1['percentual'] > 5]

display(resultado1)

resultado1 = resultado1.append(others, ignore_index=True)

labels = [str(resultado1['type'][i]) + ' '+'['+ str(round(resultado1['percentual'][i], 2)) + '%'+']' for i in resultado1.index]

#Mapa de cores
cs = cm.Set3(np.arange(100))

#Criar figura
f = plt.figure()

plt.pie(resultado1['COUNT'], labeldistance=1, radius=3, colors=cs, wedgeprops=dict(width=0.8))
plt.legend(labels = labels, loc = 'center', prop = {'size':12})


Calculando o numero de titulos por Generos:

In [ ]:
sql_consulta_genero = "SELECT genres, COUNT(*) FROM titles WHERE type = 'movie' GROUP BY genres"

resultado2 = pd.read_sql_query(sql_consulta_genero, conn)

display(resultado2)


,genres,COUNT(*)
0,Action,14186
1,"Action,Adult",11
2,"Action,Adult,Adventure",2
3,"Action,Adult,Comedy",6
4,"Action,Adult,Crime",9
...,...,...
1461,"Thriller,Western",43
1462,War,1314
1463,"War,Western",14
1464,Western,5146


In [16]:
resultado2['genres'] = resultado2['genres'].str.lower().values

#Exclui valores sem genero
temp = resultado2['genres'].dropna()

display(temp)

padrao = '(?u)\b[\\w-]+\\b'

vetor = CountVectorizer(token_pattern = padrao, analyzer = 'word').fit(temp)

0                       action
1                 action,adult
2       action,adult,adventure
3          action,adult,comedy
4           action,adult,crime
                 ...          
1461          thriller,western
1462                       war
1463               war,western
1464                   western
1465                        \n
Name: genres, Length: 1466, dtype: object

ValueError: ignored